# Empty

Empty template for your purpose.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import random

from qnt.data import ds, load_assets, load_data, write_output, restore_origin_data
from qnt.stepper import test_strategy
from qnt.stats import calc_stat
from qnt.graph import make_plot, make_plot_double, make_plot_filled

In [2]:
assets = load_assets()
ids = [a['id'] for a in assets]
ids = ids[-100:]
len(ids)

100

In [3]:
data = load_data(min_date="2010-01-01", max_date=None, dims=('time', 'field', 'asset'), assets=ids)

fetched chunk 2010-09-04 -> 2018-11-20 6s
fetched chunk 2010-01-01 -> 2010-09-03 7s
Data loaded 7s


In [4]:
def step(data):
    assets = data.sel(field='open').isel(time=0).dropna('asset').coords['asset']
    pct = 1./len(assets)
    return xr.DataArray(
        np.full([len(assets)], pct, dtype = np.float64),
        dims = [ds.ASSET],
        coords = {ds.ASSET:assets}
    )

In [5]:
output = test_strategy(data, step=step, init_data_length=30)

Testing started...
Testing progress: 1578/2208 5s
Testing complete 7.258277893066406s


In [6]:
stat = calc_stat(data, output)
stat.to_pandas().tail()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments
time,,,,,,,,,
2018-11-14,3.059284,-0.002983,0.120407,-0.111316,0.290508,0.219476,0.019333,1.0,99.0
2018-11-15,3.073298,0.004581,0.120332,-0.107245,0.290508,0.307817,0.030218,1.0,99.0
2018-11-16,3.067539,-0.001874,0.120054,-0.108918,0.290508,0.221815,0.019584,1.0,99.0
2018-11-19,3.029309,-0.012463,0.120707,-0.120023,0.290508,0.113263,0.006378,1.0,99.0
2018-11-20,2.992988,-0.011990,0.121198,-0.130574,0.290508,-0.027824,-0.010688,1.0,75.0


In [7]:
write_output(output)

write output: /home/username/fractions.nc.gz
